### Import Library

In [1]:
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import argparse
import c3d
import importlib
from typing import List

# Import helpers
sys.path.append('/app/hpe-core/datasets/vicon_processing/v2')
import helpers

# Import bimvee
sys.path.append('./submodules/bimvee')
from bimvee.importIitYarp import importIitYarp

### Load parameters

In [ ]:
vicon_c3d_file: str = '/data/EventLS/calib_test1/VICON/test10_calib02.c3d'
events_folder: str = '/data/EventLS/calib_test1/Events/right'
calib_path: str = '/data/EventLS/Calibration/calib_right.txt'
period: float = 1.0 / 25
label_tag_file: str = '../scripts/config/labels_tags_vicon.yml'

### Import event data

In [3]:
v_data: dict = importIitYarp(filePathOrName=events_folder, template=None)
e_ts: np.ndarray = v_data['data']['left']['dvs']['ts']
e_us: np.ndarray = v_data['data']['left']['dvs']['x']
e_vs: np.ndarray = v_data['data']['left']['dvs']['y']

importIitYarp trying path: /data/EventLS/calib_test2/Events/left


100%|██████████| 14875/14875 [00:37<00:00, 399.88it/s] 


Examining info.log: /data/EventLS/calib_test2/Events/left/info.log


### Import C3D data

In [ ]:
c3d_data = c3d.Reader(open(vicon_c3d_file, 'rb'))
points_3d: dict = {}
for i, points, analog in c3d_data.read_frames():
    points_3d[i] = points

marker_t = np.linspace(0.0, c3d_data.frame_count / c3d_data.point_rate, 
                            c3d_data.frame_count, endpoint=False)

In [ ]:
points_3d[1]

### Import calibration data

In [ ]:
calib = np.genfromtxt(calib_path, delimiter=" ", skip_header=1, dtype=object)

calib_dict = {}
keys = calib[:, 0].astype(str)
vals = calib[:, 1].astype(float)
calib_dict = {
    key: value for key, value in zip(keys, vals)
}
cam_res = np.int64([calib_dict['h'], calib_dict['w']])
cam_res_rgb = np.append(cam_res, 3)
# intrinsic
K = np.array([
    [calib_dict['fx'], 0.0, calib_dict['cx']],
    [0.0, calib_dict['fy'], calib_dict['cy']],
    [0.0, 0.0, 1.0]
])
# camera distortion
D = np.array([calib_dict['k1'], calib_dict['k2'], calib_dict['p1'], calib_dict['p2']])


### Visualize event data as time window

In [ ]:
img = np.ones(cam_res, dtype = np.uint8)*255

ft = e_ts[0]
for i in range(0,len(e_ts)):
    #clear the frame at intervals
    if e_ts[i] >= ft:
        cv2.imshow('Visualisation', img)
        k = cv2.waitKey(np.int64(period*1000))
        if k == 27:  # ESC key to stop
            break
        img = np.ones(cam_res, dtype = np.uint8)*255
        ft = ft + period
    img[e_vs[i],e_us[i]] = 0
    
cv2.destroyAllWindows()

### Label sequence

In [ ]:
importlib.reload(helpers)
from helpers import DvsLabeler

labels_path = os.path.join(os.path.dirname(vicon_c3d_file), "labels_all.yml")

time_tags, event_indices = helpers.calc_indices(e_ts, period)

img = np.ones(cam_res, dtype = np.uint8)*255

labeler = DvsLabeler(img_shape=(720, 1280, 3), subject='box')
labeler.label_data(e_ts, e_us, e_vs, event_indices, time_tags, period, label_tag_file)
labeler.save_labeled_points(labels_path)

print("Saved labeled points at:", labels_path)
    
cv2.destroyAllWindows()

In [ ]:
time_tags, event_indices = helpers.calc_indices(e_ts, period)

### Load labels and calculate transformation matrix

In [ ]:
importlib.reload(helpers)
from helpers import ViconHelper
from scipy.spatial.transform import Rotation as R

time_tags, event_indices = helpers.calc_indices(e_ts, period)
e_tags = e_ts[event_indices]
delay = e_tags[-1] - marker_t[-1]   # get time difference so to generate ids for the frames
print("Delay: ", delay)

labels_path = os.path.join(os.path.dirname(vicon_c3d_file), "labels3.yml")

labeled_points = helpers.read_points_labels(labels_path) # read labeled points from the yaml file
labels_time = labeled_points['times']

vicon_helper = ViconHelper(marker_t, points_3d, delay, c3d_data.frame_count, c3d_data.point_rate, c3d_data.point_labels, True, True) # if needed create the function to call instead of None

# extract the frames_id from the labeled points
frames_id = vicon_helper.get_frame_time(labeled_points['times'])
vicon_points = vicon_helper.get_vicon_points_interpolated(labeled_points)
print("First few label dicts from YAML:")
for i, d in enumerate(labeled_points['points']):
    print(f"Frame {i}: {d}")

# Solve PnP
world_points = []
image_points = []

for dvs_frame, vicon_frame in zip(labeled_points['points'], vicon_points['points']):
    labels = dvs_frame.keys()

    for l in labels:
        try:
            w_p = vicon_frame[l]
            i_p = [
                dvs_frame[l]['x'],
                dvs_frame[l]['y'] #,
                # 1.0
            ]

            world_points.append(w_p)
            image_points.append(i_p)
        except Exception as e:
            print("The stored image labels probably don't match with the vicon labels used.")
            print(e)

world_points = np.array(world_points, dtype=np.float64)
image_points = np.array(image_points, dtype=np.float64)
image_points = image_points[:, :2]

print("world_points shape:", world_points.shape)
print("image_points shape:", image_points.shape)

# Solve PnP, with 3D points from vicon and 2D points from labels
success, rvec, tvec = cv2.solvePnP(world_points, image_points, K, D)

# Calculate transformation matrix from system to camera
if success:
    R_w2c, _ = cv2.Rodrigues(rvec)
    T_w2c = np.zeros((4, 4))
    T_w2c[0:3, 0:3] = R_w2c
    T_w2c[0:3, 3] = tvec[:, 0]
    T_w2c[3, 3] = 1.0
    T_world_to_system = vicon_helper.compute_camera_marker_transforms(c3d_data, points_3d)
    frame_id = vicon_points['frame_ids'][0]
    T_system_to_camera = T_w2c @ np.linalg.inv(T_world_to_system[frame_id])
    print("Transformation matrix from system to camera:")
    print(T_system_to_camera)

### Debug: Create averaged transformation matrix from system to camera

In [ ]:
importlib.reload(helpers)
from helpers import ViconHelper
from scipy.spatial.transform import Rotation as R

time_tags, event_indices = helpers.calc_indices(e_ts, period)
e_tags = e_ts[event_indices]
delay = e_tags[-1] - marker_t[-1]   # get time difference so to generate ids for the frames
print("Delay: ", delay)

labels_path = os.path.join(os.path.dirname(vicon_c3d_file), "labels_all.yml")

labeled_points = helpers.read_points_labels(labels_path) # read labeled points from the yaml file
labels_time = labeled_points['times']

vicon_helper = ViconHelper(marker_t, points_3d, delay, c3d_data.frame_count, c3d_data.point_rate, c3d_data.point_labels, True, True) # if needed create the function to call instead of None

# extract the frames_id from the labeled points
frames_id = vicon_helper.get_frame_time(labeled_points['times'])
vicon_points = vicon_helper.get_vicon_points_interpolated(labeled_points)
print("First few label dicts from YAML:")
for i, d in enumerate(labeled_points['points']):
    print(f"Frame {i}: {d}")

# Solve PnP
world_points = []
image_points = []
Ts_system_to_camera: List[np.ndarray] = []
Ts_world_to_system = vicon_helper.compute_camera_marker_transforms(c3d_data, points_3d)

for idx, (dvs_frame, vicon_frame) in enumerate(zip(labeled_points['points'], vicon_points['points'])):
    labels = dvs_frame.keys()

    for l in labels:
        try:
            w_p = vicon_frame[l]
            i_p = [
                dvs_frame[l]['x'],
                dvs_frame[l]['y'] #,
                # 1.0
            ]

            world_points.append(w_p)
            image_points.append(i_p)
        except Exception as e:
            print("The stored image labels probably don't match with the vicon labels used.")
            print(e)

    world_points = np.array(world_points, dtype=np.float64)
    image_points = np.array(image_points, dtype=np.float64)
    image_points = image_points[:, :2]


    # Solve PnP, with 3D points from vicon and 2D points from labels
    success, rvec, tvec = cv2.solvePnP(world_points, image_points, K, D)

    # Calculate transformation matrix from system to camera
    if success:
        R_w2c, _ = cv2.Rodrigues(rvec)
        T_w2c = np.zeros((4, 4))
        T_w2c[0:3, 0:3] = R_w2c
        T_w2c[0:3, 3] = tvec[:, 0]
        T_w2c[3, 3] = 1.0
        frame_id = vicon_points['frame_ids'][idx]
        T_system_to_camera = T_w2c @ np.linalg.inv(Ts_world_to_system[frame_id])
        Ts_system_to_camera.append(T_system_to_camera)

### Debug: Use scipy optimizer to get Transformation matrix from system to camera

In [ ]:
from scipy.optimize import least_squares

def reprojection_error(params, Ps, pc, K, dist):
    """
    params: 6次元 (rotation_vector [3], translation [3])
    Ps: (N, 3) 3D点（システム座標系）
    pc: (N, 2) 2D点（画像座標系）
    K: (3, 3) カメラ行列
    dist: (5,) 歪みパラメータ（OpenCV形式）
    """
    rvec = params[:3]
    tvec = params[3:6]

    # OpenCVのprojectPointsで再投影
    projected_points, _ = cv2.projectPoints(Ps, rvec, tvec, K, dist)
    projected_points = projected_points.reshape(-1, 2)

    return (projected_points - pc).ravel()  # フラット化して返す

def estimate_Tstoc(Ps, pc, K, dist, init_params=None):
    """
    Ps: (N, 3) 3D点（システム座標系）
    pc: (N, 2) 2D点（画像座標系）
    K: (3, 3) 内部パラメータ
    dist: (5,) 歪み
    return: (4, 4) システム→カメラ座標変換行列 Tstoc
    """

    # 初期値: ゼロ回転、ゼロ並進
    if init_params is None:
        # 初期値をゼロに設定
        init_params = np.zeros(6)

    # 最小化（LM法などを選択可）
    res = least_squares(
        reprojection_error,
        init_params,
        args=(Ps, pc, K, dist),
        method='lm'  # Levenberg-Marquardt
    )

    rvec_opt = res.x[:3]
    tvec_opt = res.x[3:6]
    R_opt, _ = cv2.Rodrigues(rvec_opt)

    # 同次変換行列を構成
    T = np.eye(4)
    T[:3, :3] = R_opt
    T[:3, 3] = tvec_opt

    return T, res

In [ ]:
importlib.reload(helpers)
from helpers import ViconHelper

time_tags, event_indices = helpers.calc_indices(e_ts, period)
e_tags = e_ts[event_indices]
delay = e_tags[-1] - marker_t[-1]   # get time difference so to generate ids for the frames
print("Delay: ", delay)

labels_path = os.path.join(os.path.dirname(vicon_c3d_file), "labels_all.yml")

labeled_points = helpers.read_points_labels(labels_path) # read labeled points from the yaml file
labels_time = labeled_points['times']

vicon_helper = ViconHelper(marker_t, points_3d, delay, c3d_data.frame_count, c3d_data.point_rate, c3d_data.point_labels, True, True) # if needed create the function to call instead of None

# extract the frames_id from the labeled points
frames_id = vicon_helper.get_frame_time(labeled_points['times'])
vicon_points = vicon_helper.get_vicon_points_interpolated(labeled_points)
print("First few label dicts from YAML:")
for i, d in enumerate(labeled_points['points']):
    print(f"Frame {i}: {d}")

# Solve PnP
system_points = []
image_points = []
Ts_system_to_camera: List[np.ndarray] = []
Ts_world_to_system = vicon_helper.compute_camera_marker_transforms(c3d_data, points_3d)

for idx, (dvs_frame, vicon_frame) in enumerate(zip(labeled_points['points'], vicon_points['points'])):
    labels = dvs_frame.keys()

    for l in labels:
        try:
            w_p = vicon_frame[l]
            w_ph = np.append(w_p, 1.0)  # homogeneous coordinates
            i_p = [
                dvs_frame[l]['x'],
                dvs_frame[l]['y'] #,
                # 1.0
            ]
            frame_id = vicon_points['frame_ids'][idx]
            p_sys = Ts_world_to_system[frame_id] @ w_ph  # convert to system coordinates
            system_points.append(p_sys[:3])
            image_points.append(i_p)
        except Exception as e:
            print("The stored image labels probably don't match with the vicon labels used.")
            print(e)

system_points = np.array(system_points, dtype=np.float64)
image_points = np.array(image_points, dtype=np.float64)
image_points = image_points[:, :2]



In [ ]:
init_T = np.array([[ 5.54118388e-01, -6.46688971e-01, 5.24162298e-01, -1.25350531e+02], [-3.65516271e-01, -7.54740636e-01, -5.44760888e-01, -8.06780263e+00], [ 7.47897430e-01, 1.10272191e-01, -6.54591006e-01, -9.41323412e+01], [ 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]) 

In [ ]:
from scipy.spatial.transform import Rotation
init_T
r_vec = Rotation.from_matrix(init_T[:3, :3]).as_rotvec()
t_vec = init_T[:3, 3]
r_vec, t_vec, init_T
init_param = np.concatenate((r_vec, t_vec))
init_param

In [ ]:
T_w2s_opt, res = estimate_Tstoc(system_points, image_points, K, D, init_param)
T_w2s_opt, res

### Project points in dynamic scene

In [ ]:
marker_names = ['stereoatis:cam_right', 'stereoatis:cam_back', 'stereoatis:cam_left',
                'box:top_right_Origin', 'box:bottom_right', 'box:bottom_left', 'box:top_left']
projected_points = helpers.project_vicon_to_event_plane_dynamic(
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, D=D
)

### Debug

In [ ]:
e_ts[-1]

In [ ]:
marker_t[-2]

In [ ]:
Ts_system_to_camera = []

In [ ]:
Ts_system_to_camera.append(T_system_to_camera)

In [ ]:
Ts_system_to_camera

In [ ]:
T_system_to_camera_ave = helpers.average_transforms(Ts_system_to_camera)
T_system_to_camera_ave

In [ ]:
importlib.reload(helpers)
T_system_to_camera_ave_no_outlier = helpers.average_transforms_with_outlier_removal(Ts_system_to_camera, 0.1)
T_system_to_camera_ave_no_outlier

In [ ]:
marker_names = ['stereoatis:cam_right', 'stereoatis:cam_back', 'stereoatis:cam_left',
                'box:top_right_Origin', 'box:bottom_right', 'box:bottom_left', 'box:top_left']
projected_points = helpers.project_vicon_to_event_plane_dynamic(
    marker_names, c3d_data, points_3d, marker_t, T_system_to_camera_ave_no_outlier, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, D=D
)

In [ ]:
marker_names = ['stereoatis:cam_right', 'stereoatis:cam_back', 'stereoatis:cam_left',
                'box:top_right_Origin', 'box:bottom_right', 'box:bottom_left', 'box:top_left']
xyz = 1
ps = helpers.marker_p(c3d_data.point_labels, points_3d.values(), marker_names[0], None)
plt.plot(ps[:, xyz])
ps = helpers.marker_p(c3d_data.point_labels, points_3d.values(), marker_names[1], None)
plt.plot(ps[:, xyz])
ps = helpers.marker_p(c3d_data.point_labels, points_3d.values(), marker_names[2], None)
plt.plot(ps[:, xyz])

In [ ]:
tmp = [T[0, 3] for T in Ts_world_to_system]
plt.plot(tmp)